In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('sample_db.sqlite')

In [ ]:
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE NODE_INFO (
nodeName VARCHAR(50) PRIMARY KEY,
osVersion VARCHAR(50),
nodeExporterStatusInstalled BOOLEAN,
nodeExporterStatus VARCHAR(50),
googleServiceStatus VARCHAR(50),
consulServiceStatus VARCHAR(50),
ntpStatus VARCHAR(50),
systemType VARCHAR(50),
uptime VARCHAR(10),
logsize VARCHAR(50),
appVersion VARCHAR(50),
splunkStatusInstalled BOOLEAN,
splunkStatus VARCHAR(50),
splunkStatusVersion VARCHAR(50),
waitTime VARCHAR(50),
certPackageOps VARCHAR(50),
certPackageQtfa VARCHAR(50),
bootstrapInfoVersion VARCHAR(50),
bootstrapInfoDate VARCHAR(50),
sssdInfoStatus VARCHAR(50),
sssdInfoUptime VARCHAR(100)
)
''')

conn.commit()

In [ ]:
import json
f = open('/content/test_data_sample_2.json')

# returns JSON object as
# a dictionary
data = json.load(f)
preprocessed =[]
for i in data['node_data']:
  app_version_data = data['node_data'][i]['app_version']
  if isinstance(app_version_data, list):
    app_version_str = ", ".join(app_version_data)
  elif isinstance(app_version_data, str):
    app_version_str = app_version_data
  else:
    app_version_str = str(app_version_data)
  preprocessed.append((i,data['node_data'][i]['os_version'],data['node_data'][i]['node_exporter_status']['installed'],data['node_data'][i]['node_exporter_status']['status'],data['node_data'][i]['google_service_status'],data['node_data'][i]['consul_service_status'],data['node_data'][i]['ntp_status'],data['node_data'][i]['system_type'],data['node_data'][i]['uptime'],data['node_data'][i]['logsize'],app_version_str,data['node_data'][i]['splunk_status']['installed'],data['node_data'][i]['splunk_status']['status'],data['node_data'][i]['splunk_status']['version'],data['node_data'][i]['wait_time'],data['node_data'][i]['cert_package']['ops'],data['node_data'][i]['cert_package']['qtfa'],data['node_data'][i]['bootstrap_info']['Bootstrap_Version'],data['node_data'][i]['bootstrap_info']['Date'],data['node_data'][i]['sssd_info']['status'],data['node_data'][i]['sssd_info']['uptime']))

# Closing file
f.close()


In [ ]:
print (data)

{'generated_on': '2024-03-22T12:33:45.120505', 'version': '1.0.2', 'general_pod_details': {'pod_number': 'c90', 'datacenter': 'raleigh', 'oel_unreachable_host_date': '2024/03/22/11:59', 'total_nodes': 128, 'ansible_unreachable_nodes': 5, 'reachable_nodes': 121, 'nodes_with_login_issues': 2, 'os_details': {'Oracle Linux Server release 9.2': 3, 'CentOS Linux release 7.6.1810 (Core)': 8, 'Oracle Linux Server release 7.9': 91, 'CentOS release 6.10 (Final)': 3, 'CentOS Linux release 7.9.2009 (Core)': 15, 'Oracle Linux Server release 8.8': 1}, 'node_exporter_status': {'installed_active': 65, 'installed_inactive': 0, 'not_installed': 56}, 'nodes_not_on_prom': 0, 'google_service_status': {'unknown_app': 44, 'Running': 73, 'Down': 4}, 'consul_service_status': {'not_installed': 37, 'running': 83, 'down': 1}, 'kafka_status': {'misconfigured_topic_count': 0, 'misconfigured_topics': [], 'smoke_test_passed': 2, 'smoke_test_failed': 0, 'under_replicated_partitions': 0}, 'ntp_status': {'undefined': 9,

In [ ]:
cursor.executemany('''
INSERT INTO NODE_INFO (nodeName, osVersion, nodeExporterStatusInstalled,nodeExporterStatus,googleServiceStatus,consulServiceStatus,ntpStatus,systemType,uptime,logsize,appVersion,splunkStatusInstalled, splunkStatus,splunkStatusVersion,waitTime,certPackageOps,certPackageQtfa,bootstrapInfoVersion,bootstrapInfoDate,sssdInfoStatus,sssdInfoUptime) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?,?,?,?, ?, ?,?,? ,?)
''', preprocessed)

In [ ]:
query = 'SELECT * FROM NODE_INFO'

In [ ]:
cursor.execute(query)
rows = cursor.fetchall()

In [ ]:
print (rows)

[('ac01.c90.eng.raleigh.google.com', 'Oracle Linux Server release 9.2', 0, 'N/A', 'unknown_app', 'not_installed', 'undefined', 'CloudStack KVM Hypervisor', '133', '0B', 'tigervnc-server-1.12.0-13.el9_2.x86_64', 0, 'N/A', 'N/A', '0', 'ops-ca-bundle-1.0-8.noarch', 'qtfa-1.0-44.noarch', '1.0.0', '2023-07-10T20', 'active', 'Wed 2024-03-13 17:30:03 UTC; 1 week 1 day ago'), ('camsrepo01.c90.eng.raleigh.google.com', 'CentOS Linux release 7.6.1810 (Core)', 1, 'active', 'Running', 'not_installed', 'in-sync', 'VMware Virtual Platform', '144', '0B', 'cams_registry-2.3.0-6.noarch', 1, 'active', 'splunk-agent-1.0.1-15.el7.x86_64', '0', 'ops-ca-bundle-1.0-8.noarch', 'qtfa-1.0-44.noarch', 'UNKNOWN', 'UNKNOWN', 'active', 'Wed 2024-03-13 17:30:03 UTC; 1 weeks 1 days ago'), ('cassdb01.c90.eng.raleigh.google.com', 'Oracle Linux Server release 7.9', 0, 'N/A', 'Running', 'running', 'in-sync', 'CloudStack KVM Hypervisor', '168', '3.0MiB', '3.11.10', 1, 'active', 'splunk-agent-1.0.1-15.el7.x86_64', '0', 'ops

Now that Data is in the DB, lets begin the real work

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 33.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
torch.cuda.is_available()

True

In [ ]:
available_memory = torch.cuda.get_device_properties(0).total_memory

print(available_memory)

15835660288


Action time

In [ ]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        load_in_8bit=True,
        # torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE NODE_INFO (
nodeName VARCHAR(50) PRIMARY KEY, -- Unique Name for each node
osVersion VARCHAR(50), -- Name of the OS version of the node
nodeExporterStatusInstalled BOOLEAN, -- Installation status of the node exporter of the node
nodeExporterStatus VARCHAR(50), -- exporter status of the node
googleServiceStatus VARCHAR(50), -- google service status of the node
consulServiceStatus VARCHAR(50), -- consul service status of the node
ntpStatus VARCHAR(50), -- ntp status of the node
systemType VARCHAR(50), -- system type of the node
uptime VARCHAR(10), -- uptime of the node
logsize VARCHAR(50), -- log size of the node
appVersion VARCHAR(50), -- app version of the node
splunkStatusInstalled Boolean, -- splunk install status of the node,
splunkStatus VARCHAR(50), -- splunk status of the node,
splunkStatusVersion VARCHAR(50), -- splunk status version of the node
waitTime VARCHAR(50), -- wait time of the node
certPackageOps VARCHAR(50), -- cert package ops of the node
certPackageQtfa VARCHAR(50), -- cert package qtfa of the node
bootstrapInfoDate VARCHAR(50), -- bootstrap information date of the node
sssdInfoStatus VARCHAR(50), -- sssd information status of the node
sssdInfoUptime VARCHAR(50), -- sssd information uptime of the node
)

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [ ]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [ ]:
question = "What is the bootstrap version of each node?"
generated_sql = generate_query(question)

In [ ]:
print(generated_sql)


SELECT ni.nodeName,
       ni.bootstrapInfoDate
FROM NODE_INFO ni
ORDER BY ni.nodeName NULLS LAST;


In [ ]:
cursor.execute(generated_sql)
rows = cursor.fetchall()

In [ ]:
print(rows)

[('ac01.c90.eng.raleigh.google.com', '2023-07-10T20'), ('camsrepo01.c90.eng.raleigh.google.com', 'UNKNOWN'), ('cassdb01.c90.eng.raleigh.google.com', '2023-10-06T07'), ('cassdb02.c90.eng.raleigh.google.com', '2023-10-06T06'), ('cassdb03.c90.eng.raleigh.google.com', '2023-10-06T06'), ('cassdb04.c90.eng.raleigh.google.com', '2023-10-06T06'), ('cassdb05.c90.eng.raleigh.google.com', '2023-10-06T06'), ('comp01.c90.eng.raleigh.google.com', 'UNKNOWN'), ('ctrl01.c90.eng.raleigh.google.com', '2020-09-11T19'), ('ds01.c90.eng.raleigh.google.com', '2024-03-22T00'), ('ds02.c90.eng.raleigh.google.com', '2023-02-24T00'), ('dst01.c90.eng.raleigh.google.com', 'UNKNOWN'), ('dst04.c90.eng.raleigh.google.com', '2023-08-08T13'), ('dwn01.c90.eng.raleigh.google.com', 'UNKNOWN'), ('dwn02.c90.eng.raleigh.google.com', 'UNKNOWN'), ('ioc02.c90.eng.raleigh.google.com', 'UNKNOWN'), ('itamcatalog01.c90.eng.raleigh.google.com', '2023-07-14T17'), ('itamcatalog02.c90.eng.raleigh.google.com', '2024-03-07T21'), ('jdc-kafk